In [ ]:
from scipy import stats
import pandas as pd
import numpy as np
from scipy.stats import norm
import yfinance as yf
from ipywidgets import interact, fixed, IntSlider, Dropdown, Tab, Output
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go


In [ ]:
# Define the list of funds
funds = ["IVV", "IYW", "IYF", "IYZ", "IYH", "IYK", "IYE", "IYJ", "IDU", "IYM", "IYC", "IYR"]



In [ ]:
# Fetch the fund data
data = {}
for fund in funds:
    data[fund] = yf.download(fund, start="2000-01-01", end="2023-06-22")


In [ ]:
# Define the function to compute the drawdowns
def compute_drawdowns(data_series):
    # Calculate the running maximum.
    running_max = data_series.cummax()
    # Calculate the drawdowns.
    drawdowns = -1 * (running_max - data_series) / running_max
    return drawdowns


In [ ]:
# Define a function that plots the rolling correlation between two funds over a specified window
def plot_rolling_correlation(fund_a, fund_b, window):
    # Get the 'Close' price data for both funds
    price_a = data[fund_a]['Close']
    price_b = data[fund_b]['Close']
    
    # Drop any missing values from the price data
    price_a = price_a.dropna()
    price_b = price_b.dropna()
    
    # Calculate the rolling correlation over the specified window. This is done by first creating a rolling 
    # window on the 'price_a' series, and then applying the correlation (.corr()) method with the 'price_b' series. 
    rolling_corr = price_a.rolling(window=window).corr(price_b)

    # Initialize a plotly graph object Figure
    fig = go.Figure()
    
    # Add a trace (a graphical representation of data in the figure) to the figure. 
    # Here, we're adding a line plot of the rolling correlation.
    # 'x' and 'y' are set to the dates and the calculated rolling correlation values respectively. 
    fig.add_trace(go.Scatter(x=rolling_corr.index, y=rolling_corr.values, mode='lines', name='correlation'))
    
    # Update the layout of the figure to add a title, and labels for the x-axis and y-axis.
    fig.update_layout(title=f'Rolling Correlation of {fund_a} and {fund_b} over a {window}-day window', xaxis_title='Date', yaxis_title='Rolling Correlation')
    
    # Display the figure
    fig.show()


In [ ]:
# Define a function that plots the Cumulative Distribution Function (CDF) of rolling correlations for all combinations of funds
def plot_cdf(window):
    # Initialize an empty list to hold correlation values
    correlations = []

    # Iterate over all combinations of two distinct funds (since correlation is a pairwise measure)
    for i in range(len(funds)):
        for j in range(i+1, len(funds)):
            # Get the names of the two funds
            fund_a = funds[i]
            fund_b = funds[j]
            
            # Extract the closing prices for these funds
            price_a = data[fund_a]['Close']
            price_b = data[fund_b]['Close']
            
            # Drop any missing values from the price data
            price_a = price_a.dropna()
            price_b = price_b.dropna()
            
            # Compute the rolling correlation between these two funds over the specified window
            rolling_corr = price_a.rolling(window=window).corr(price_b)
            
            # Add the correlations to the list (after dropping missing values), essentially 'flattening' the data into one list
            correlations.extend(rolling_corr.dropna().values)

    # Compute the cumulative frequencies of the correlations using scipy's cumfreq function with 1000 bins
    res = stats.cumfreq(correlations, numbins=1000)
    
    # Compute the x-values for the CDF plot. The x-values are the lower limit of the bins plus a linspace the size of the cumulative count array.
    x = res.lowerlimit + np.linspace(0, res.binsize*res.cumcount.size, res.cumcount.size)

    # Initialize a plotly graph object Figure
    fig = go.Figure()
    
    # Add a trace to the figure for the CDF. The y-values are the cumulative count divided by the total number of correlations, 
    # which gives the CDF. The mode is set to 'lines' to create a line plot.
    fig.add_trace(go.Scatter(x=x, y=res.cumcount / len(correlations), mode='lines', name='CDF'))
    
    # Update the layout of the figure to add a title, and labels for the x-axis and y-axis.
    fig.update_layout(title='CDF of rolling correlations', xaxis_title='Correlation', yaxis_title='CDF')
    
    # Display the figure
    fig.show()


In [ ]:
def drawdown_table(funds, data):
    drawdown_data = []
    for fund in funds:
        # get the price data for this fund
        price = data[fund]['Close']

        # We calculate the drawdowns for all the funds.
        drawdowns = compute_drawdowns(price)

        # filter the drawdowns to only include those that are greater than 10%
        filtered_drawdowns = drawdowns[drawdowns <= -0.10]

        if not filtered_drawdowns.empty:
            # get the date of the maximum drawdown
            max_drawdown_date = filtered_drawdowns.idxmin()

            # get the recovery date
            recovery_date = drawdowns[max_drawdown_date:][drawdowns[max_drawdown_date:] >= 0].index[0] if any(drawdowns[max_drawdown_date:] >= 0) else None

            # calculate the length of the drawdown
            drawdown_length = (max_drawdown_date - filtered_drawdowns.index[0]).days

            # calculate the length of the recovery
            recovery_length = (recovery_date - max_drawdown_date).days if recovery_date else None

            # calculate the total underwater period
            underwater_period = (recovery_date - filtered_drawdowns.index[0]).days if recovery_date else None

            # append the data to our list
            drawdown_data.append([fund, filtered_drawdowns.index[0], max_drawdown_date, drawdown_length, recovery_date, recovery_length, underwater_period, filtered_drawdowns.min()])

    # create a DataFrame from our list
    drawdown_df = pd.DataFrame(drawdown_data, columns=['Fund', 'Start', 'End', 'Length', 'Recovery By', 'Recovery Time', 'Underwater Period', 'Drawdown'])

    # convert the timedelta columns to a more readable format
    drawdown_df['Length'] = drawdown_df['Length'].apply(lambda x: f"{divmod(x, 365.25)[0]} years {divmod(divmod(x, 365.25)[1], 30.44)[0]} months") 
    drawdown_df['Recovery Time'] = drawdown_df['Recovery Time'].apply(lambda x: f"{divmod(x, 365.25)[0]} years {divmod(divmod(x, 365.25)[1], 30.44)[0]} months" if pd.notnull(x) else None)
    drawdown_df['Underwater Period'] = drawdown_df['Underwater Period'].apply(lambda x: f"{divmod(x, 365.25)[0]} years {divmod(divmod(x, 365.25)[1], 30.44)[0]} months" if pd.notnull(x) else None)

    # convert the drawdowns to a percentage, rounding to the nearest hundredth place and adding a percent sign
    drawdown_df['Drawdown'] = round((drawdown_df['Drawdown'] * 100), 2).map("{:.2f}%".format)

    # sort the DataFrame by drawdown and add a rank column
    drawdown_df = drawdown_df.sort_values(by='Drawdown')
    drawdown_df = drawdown_df.reset_index(drop=True)
    drawdown_df.index += 1

    return drawdown_df

In [ ]:
# Cell 6: Create widgets and tabs
fund_a_dropdown = Dropdown(options=funds, description='Fund A:')
fund_b_dropdown = Dropdown(options=funds, description='Fund B:')
window_slider = IntSlider(min=1, max=365, step=1, value=30, description='Window:')
tab = Tab()

# Create Output widgets
out1 = Output()
out2 = Output()
out3 = Output()

# Add the Output widgets to the Tab widget
tab.children = [out1, out2, out3]
tab.set_title(0, 'Rolling Correlation')
tab.set_title(1, 'CDF of Correlations')
tab.set_title(2,'Fund Drawdowns')

# Define update functions
def update_rolling_corr(*args):
    out1.clear_output()
    with out1:
        plot_rolling_correlation(fund_a_dropdown.value, fund_b_dropdown.value, window_slider.value)

def update_cdf(*args):
    out2.clear_output()
    with out2:
        plot_cdf(window_slider.value)

def update_drawdown_table(*args):
    out3.clear_output()
    with out3:
        display(drawdown_table(funds, data))  # Pass 'funds' and 'data' as arguments to 'drawdown_table()'

# Observe changes in dropdowns and slider
fund_a_dropdown.observe(update_rolling_corr, 'value')
fund_b_dropdown.observe(update_rolling_corr, 'value')
window_slider.observe(update_rolling_corr, 'value')
window_slider.observe(update_cdf, 'value')

# Initial displays
update_rolling_corr()
update_cdf()
update_drawdown_table()

# Display widgets
display(fund_a_dropdown, fund_b_dropdown, window_slider, tab)